<span style='font-size: 40px;'> Lending Club - ML Modeling with LightGBM </span>



<span style='font-size: 15px;'> Lending Club is an online peer-to-peer lending platform that allows small-dollar investors to lend borrower small unsecured personal loans. The interest rate charged on the loan is done on the basis of the grades and subgrades formulated by Lending Club. The method of calculating these grades is not transparent and the purpose of this project is to try to determine which variables are most strongly correlated with a loan being paid off by the end of its term, through a combination of statistical and visual analysis, as well as attempting to train a machine learning model to match the results or surpass those found in the datset.  </span>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.ticker import MaxNLocator
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate
import textwrap
from IPython.display import display
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,  classification_report
import lightgbm as lgb
# from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc


In [2]:
initial_df = pd.read_csv(r'LC_ML_Project\csv_files\accepted_2007_to_2018Q4.csv', encoding="ISO-8859-1")

C:\Users\97258\AppData\Local\Temp\ipykernel_19756\1839735408.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  initial_df = pd.read_csv(r'data\csv_files\accepted_2007_to_2018Q4.csv', encoding="ISO-8859-1")


<span style='font-size: 30px;'> Preliminary analysis and data cleaning </span>

There are 2.3M rows and 151 columns. That's a lot of data to parse. Let's get started!

In [4]:
initial_df.shape

(2260701, 151)

The loan status is the dependent variable we want to understand. This dataset reflects the loan status of every loan as of 2018 Q4 and includes many loans in progress. We want to simplify the analysis and keep only the loans that reached the end of their term already. This is indicated by the Fully Paid or Charged Off designations. There are additional Fully Paid and Charged Off categories, preceeded by "Does not meet the credit policy." Let's analyze whether to include these or not.

In [5]:
initial_df['loan_status'].value_counts()

loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64

Based on the below. I would like to include the Fully Paid and Charge Offs that do meet the current credit policy. The inclusion of them gives us more data to work with while not materially reducing the repayment rate, which stands at around 80%. We want to conduct statistical analysis independently without being biased by LC's credit policies.

In [27]:
print("Checking payback rate of the 'does not meets':") 
print(initial_df.loc[initial_df['loan_status'].str.contains('Does not meet the credit policy'),'loan_status'].value_counts(normalize=True))

print("\nChecking payback rate of just the regular final statuses:") 
print(initial_df.loc[initial_df['loan_status'].isin(['Fully Paid','Charged Off']),'loan_status'].value_counts(normalize=True))

print("\nChecking payback rate for all final statuses:") 
print(initial_df.loc[initial_df['loan_status'].str.contains('Fully Paid|Charged Off'),'loan_status'].value_counts(normalize=True))


Checking payback rate of the 'does not meets':
loan_status
Does not meet the credit policy. Status:Fully Paid     0.723172
Does not meet the credit policy. Status:Charged Off    0.276828
Name: proportion, dtype: float64

Checking payback rate of just the regular final statuses:
loan_status
Fully Paid     0.800374
Charged Off    0.199626
Name: proportion, dtype: float64

Checking payback rate all final statuses:
loan_status
Fully Paid                                             0.798742
Charged Off                                            0.199219
Does not meet the credit policy. Status:Fully Paid     0.001475
Does not meet the credit policy. Status:Charged Off    0.000565
Name: proportion, dtype: float64


We continue cleaning the Loan Status column by making it a binary choice being Fully Paid or Charged Off.

In [3]:
# Dropping any NAs in the loan status column
modified_df = initial_df.dropna(subset=['loan_status'])

# Replacing categorical column with numeric values, 1 for paid, 0 for charged off
modified_df.loc[modified_df['loan_status'].str.contains('Fully Paid'), 'loan_status'] = 1
modified_df['loan_status'] = modified_df['loan_status'].replace('Charged Off',0)

# There are fully paid and charged off loans with slightly different 
modified_df['loan_status'] = modified_df['loan_status'].replace('Does not meet the credit policy. Status:Charged Off',0)

# Lastly, because only the completed loans have been converted to boolian data types, I will now coerce all non-numeric values, meaning they will be dropped

modified_df = modified_df[pd.to_numeric(modified_df['loan_status'], errors='coerce').notna()] 

C:\Users\97258\AppData\Local\Temp\ipykernel_19756\2278980541.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_df['loan_status'] = modified_df['loan_status'].replace('Charged Off',0)
C:\Users\97258\AppData\Local\Temp\ipykernel_19756\2278980541.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modified_df['loan_status'] = modified_df['loan_status'].replace('Does not meet the credit policy. Status:Charged Off',0)


We confirm below that the repayment rate after all of the data cleaning above remains at around 80%. It is. Based on the below counts, it look like 80% of the loans were fully repaid and 20% were charged-off. 80% is the mark we want to try to best. 

In [10]:
# Creating table of counts
value_cts = modified_df['loan_status'].value_counts().reset_index()
value_cts.columns = ['Final Status', 'Frequency']

# Creating table of percentages
value_perc = modified_df['loan_status'].value_counts(normalize=True)*100
value_perc = value_perc.reset_index()
value_perc.columns = ['Final Status', 'Percentage']
value_perc['Percentage'] = round(value_perc['Percentage'], 2)

# Print table of value counts
count_table = tabulate(value_cts.to_dict('records'), headers='keys', tablefmt='pretty')
print(f'Value count by final Loan Status: \n{count_table}\n')

# Print table of value percentages
perc_table = tabulate(value_perc.to_dict('records'), headers='keys', tablefmt='pretty')
print(f'Percentage by final Loan Status: \n{perc_table}')


Value count by final Loan Status: 
+--------------+-----------+
| Final Status | Frequency |
+--------------+-----------+
|      1       |  1078739  |
|      0       |  269320   |
+--------------+-----------+

Percentage by final Loan Status: 
+--------------+------------+
| Final Status | Percentage |
+--------------+------------+
|      1       |   80.02    |
|      0       |   19.98    |
+--------------+------------+


We next check for nulls and remove any columns above 50%. Because there are so many columns, many of them similar to others, we are comfortable with this strict dropoff rule at 50% even though may be columns right above the edge that could be of value.

In [9]:
perc_null = round(100*(modified_df.isnull().sum()/len(modified_df.id)), 2)
print(perc_null.loc[perc_null > 0].sort_values(ascending=False)[0:40])

member_id                                     100.00
next_pymnt_d                                   99.80
orig_projected_additional_accrued_interest     99.72
hardship_dpd                                   99.57
hardship_type                                  99.57
hardship_reason                                99.57
hardship_status                                99.57
deferral_term                                  99.57
hardship_amount                                99.57
hardship_start_date                            99.57
payment_plan_start_date                        99.57
hardship_length                                99.57
hardship_end_date                              99.57
hardship_loan_status                           99.57
hardship_payoff_balance_amount                 99.57
hardship_last_payment_amount                   99.57
sec_app_mths_since_last_major_derog            99.51
sec_app_revol_util                             98.64
sec_app_chargeoff_within_12_mths              

We have decreased our column count from 151 to 93.

In [10]:
fifty_null = perc_null[perc_null>50].index
mod_df_2 = modified_df.drop(columns=fifty_null)
print(f'# of rows, columns: {mod_df_2.shape}')

# of rows, columns: (1348059, 93)


Many of the column names are unclear. After a search, I found a dictionary provided by LC explaining each column name. I pass this to a dataframed called 'definitions' and utilizing it create a dictionary for future use.

In [32]:
# An online resource from Lending Club that defines its terms

definitions = pd.read_excel("https://resources.lendingclub.com/LCDataDictionary.xlsx")

columns_list = list(mod_df_2.columns)
columns_dict = {}

for col in columns_list:
    value = list(definitions.loc[definitions['LoanStatNew'] == col,'Description'])
    if value:
        columns_dict[col] = value[0]


I looked up the definition of some returns to get an idea of the relevance of each column. A few examples are shown below, including initial_list_status. The F and W in the columns stand for whole or fractional and indicate whether investors were given the choice of funding a portion of the loan or had to lend the full requested amount. I'm less interested in the nuances of LC's platform and am more interested in more generalizable characteristics of borrowers. Therefore, I will drop this column in due course.

In [11]:
columns_dict['initial_list_status']

'The initial listing status of the loan. Possible values are – W, F'

I created an additional dictionary to make it possible to search multiple entries that contain the same key word, in this case 'num_tl'. For two entries that contain this term in the key, the value tells us that they were "updated in past 2 months." That indicates this information would have been unavailable at issuance and therefore are irrelevant for our analysis. I will drop these columns.

In [12]:
num_tl_def = {key: value for key, value in columns_dict.items() if 'num_tl' in key}
num_tl_def

{'num_tl_120dpd_2m': 'Number of accounts currently 120 days past due (updated in past 2 months)',
 'num_tl_30dpd': 'Number of accounts currently 30 days past due (updated in past 2 months)',
 'num_tl_90g_dpd_24m': 'Number of accounts 90 or more days past due in last 24 months',
 'num_tl_op_past_12m': 'Number of accounts opened in past 12 months'}

I further searched to check if any other definitions include the word 'update' to get a hint as to which were updated after initial issuance, but I find that it's only the two original ones we stumbled on. 

In [13]:
updated_after = [key for key, value in columns_dict.items() if 'update' in value]
print(updated_after)

['num_tl_120dpd_2m', 'num_tl_30dpd']


I create a value count dictonary to make it easier to view the type of data in each column and the count of each unique value. I then run a function to see which columns have single values. These columns serve no use for our analysis since they have no predictive power, and are dropped.

In [11]:
def v_count(category):
    return mod_df_2[category].value_counts()

print(v_count('acc_now_delinq'))

acc_now_delinq
0.0     1341688
1.0        5979
2.0         306
3.0          42
4.0          10
5.0           3
6.0           1
14.0          1
Name: count, dtype: int64


In [12]:
single_values = []

for col in mod_df_2.columns:
    if len(v_count(col)) == 1:
        single_values.append(col)

print(single_values)

['pymnt_plan', 'out_prncp', 'out_prncp_inv', 'policy_code', 'hardship_flag']


In [13]:
mod_df_2 = mod_df_2.drop(columns=single_values)

After researching several fields through the methods explored above, I determined which columns can be dropped due to irrelevance to my analysis. Most would have been unavailable at issue. Others correspond closely to other columns, like fico_range_high, which is simply the fico_range_low column + 5. Others, like employment title is too subjective and varied to be of use.

In [14]:
unavailable_at_issue = ['tot_cur_bal','avg_cur_bal',  'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high',
       'last_fico_range_low','last_pymnt_d','last_credit_pull_d', 'num_tl_30dpd','num_tl_120dpd_2m', 'debt_settlement_flag']

mod_df_2 = mod_df_2.drop(columns=unavailable_at_issue)

In [15]:
# A month column may be useful. Let's create one before getting rid of the issue_date, which is too far in the past and likely reflects macroeconomic trends that won't be replicated as time progresses into the future
mod_df_2['month'] = pd.to_datetime(mod_df_2['issue_d']).dt.month 


C:\Users\97258\AppData\Local\Temp\ipykernel_19756\512945082.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  mod_df_2['month'] = pd.to_datetime(mod_df_2['issue_d']).dt.month


In [16]:
unhelpful_columns = ['url','title','zip_code','fico_range_high','funded_amnt_inv', 'emp_title', 'initial_list_status', 'id','issue_d']

mod_df_2 = mod_df_2.drop(columns=unhelpful_columns)

The dataframe has gone down from 151 to 63 now. We're getting there!

In [13]:
print(f'Number of columns remaining: {len(mod_df_2.columns)}')
print(mod_df_2.columns)

Number of columns remaining: 63
Index(['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'emp_length', 'home_ownership', 'annual_inc',
       'verification_status', 'loan_status', 'purpose', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'inq_last_6mths',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq',
       'tot_coll_amt', 'total_rev_hi_lim', 'acc_open_past_24mths',
       'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt',
       'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op',
       'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc',
       'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl',
       'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl',
       'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
       'num_tl_90g_d

One final data cleaning manner: checking for any duplicates remaining. As it turns out, there are none.

In [17]:
print(mod_df_2[mod_df_2.duplicated()])

Empty DataFrame
Columns: [loan_amnt, funded_amnt, term, int_rate, installment, grade, sub_grade, emp_length, home_ownership, annual_inc, verification_status, loan_status, purpose, addr_state, dti, delinq_2yrs, earliest_cr_line, fico_range_low, inq_last_6mths, open_acc, pub_rec, revol_bal, revol_util, total_acc, collections_12_mths_ex_med, application_type, acc_now_delinq, tot_coll_amt, total_rev_hi_lim, acc_open_past_24mths, bc_open_to_buy, bc_util, chargeoff_within_12_mths, delinq_amnt, mo_sin_old_il_acct, mo_sin_old_rev_tl_op, mo_sin_rcnt_rev_tl_op, mo_sin_rcnt_tl, mort_acc, mths_since_recent_bc, mths_since_recent_inq, num_accts_ever_120_pd, num_actv_bc_tl, num_actv_rev_tl, num_bc_sats, num_bc_tl, num_il_tl, num_op_rev_tl, num_rev_accts, num_rev_tl_bal_gt_0, num_sats, num_tl_90g_dpd_24m, num_tl_op_past_12m, pct_tl_nvr_dlq, percent_bc_gt_75, pub_rec_bankruptcies, tax_liens, tot_hi_cred_lim, total_bal_ex_mort, total_bc_limit, total_il_high_credit_limit, disbursement_method, month]
Inde